In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bmarcin (use `wandb login --relogin` to force relogin)


True

In [3]:
dev_ds = "../data/dev/"
test_ds = "../data/test/"
train_ds = "../data/train/"

notebook_path_prefix = "roberta_lm"

In [4]:
special_tokens = [
    '<url>',
    '<email>',
    '<number>',
    '<date>', 
]

In [5]:
from transformers import RobertaConfig, RobertaTokenizerFast

In [6]:
tokenizer = RobertaTokenizerFast.from_pretrained(notebook_path_prefix, max_len=512, use_fast=True)

file roberta_lm/config.json not found
file roberta_lm/config.json not found


In [7]:
tokenizer.add_special_tokens({
    'additional_special_tokens': special_tokens
})

0

In [8]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': ['<url>', '<email>', '<number>', '<date>']}

In [9]:
import pandas as pd

In [10]:
data1 = pd.read_csv("../data/dev/in.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)
# data2 = pd.read_csv("../data/test/in.tsv", delimiter='\t', header=None, encoding="utf8")
# data3 = pd.read_csv("../data/train/in.tsv", delimiter='\t', header=None, encoding="utf8")

In [11]:
labels1 = pd.read_csv("../data/dev/expected.tsv", delimiter='\t', header=None, encoding="utf8", quoting=0)
# labels2 = pd.read_csv("../data/test/expected.tsv", delimiter='\t', header=None, encoding="utf8")
# labels3 = pd.read_csv("../data/train/expected.tsv", delimiter='\t', header=None, encoding="utf8")

In [12]:
print(len(data1), len(labels1))

3427 3429


In [13]:
data1.tail()

,0
3422,"EUROPEAN COMMISSION Brussels, 16.9.2021 COM(20..."
3423,20.9.2021 EN Official Journal of the European ...
3424,"EUROPEAN COMMISSION Brussels, 4.8.2021 COM(202..."
3425,"EUROPEAN COMMISSION Brussels, 8.9.2021 COM(202..."
3426,"EUROPEAN COMMISSION Brussels, 20.9.2021 COM(20..."


In [11]:
import torch

In [12]:
unique_labels = [
    'economy',
    'law',
    'foreign policy',
    'agriculture',
    'environment',
    'social policy',
    'state',
    'public authorities',
    'taxes',
    'transport',
    'science',
    'research and technology',
    'european union',
    'work and employment',
    'health',
    'education',
    'industry',
    'sports'
]

In [13]:
labels = set([label.strip().lower() for label in labels1.iloc[1][0].split(',')]); labels

{'law', 'public authorities', 'state'}

In [33]:
class ClassificationDS(torch.utils.data.Dataset):
    def __init__(self, input_texts, input_labels, unique_labels, tokenizer):
        self.input_texts = input_texts
        self.input_labels = input_labels
        self.unique_labels = unique_labels
        self.tokenizer = tokenizer
        self.label2idx = {}
        
        for label in self.unique_labels:
            self.label2idx[label] = len(self.label2idx)
            
        print(self.label2idx)
        
    def __len__(self):
        return len(self.input_texts)
        
    def __getitem__(self, idx):
        tokenized = tokenizer(str(self.input_texts.iloc[idx][1]), truncation=True, padding="max_length", max_length=512)
        labels = set([self.label2idx[label.strip().lower()] for label in self.input_labels.iloc[idx][0].split(',')])
        
        item = {
            'input_ids': torch.tensor(tokenized['input_ids']),
            'attention_mask': torch.tensor(tokenized['attention_mask']),
            'labels': torch.zeros([len(self.label2idx)]).index_fill_(0, torch.tensor(list(labels)), 1)
        }
        
        return item

In [34]:
dev_ds = ClassificationDS(data1, labels1, unique_labels, tokenizer)

{'economy': 0, 'law': 1, 'foreign policy': 2, 'agriculture': 3, 'environment': 4, 'social policy': 5, 'state': 6, 'public authorities': 7, 'taxes': 8, 'transport': 9, 'science': 10, 'research and technology': 11, 'european union': 12, 'work and employment': 13, 'health': 14, 'education': 15, 'industry': 16, 'sports': 17}


In [35]:
test_ds = ClassificationDS(data2, labels2, unique_labels, tokenizer)

{'economy': 0, 'law': 1, 'foreign policy': 2, 'agriculture': 3, 'environment': 4, 'social policy': 5, 'state': 6, 'public authorities': 7, 'taxes': 8, 'transport': 9, 'science': 10, 'research and technology': 11, 'european union': 12, 'work and employment': 13, 'health': 14, 'education': 15, 'industry': 16, 'sports': 17}


In [36]:
train_ds = ClassificationDS(data3, labels3, unique_labels, tokenizer)

{'economy': 0, 'law': 1, 'foreign policy': 2, 'agriculture': 3, 'environment': 4, 'social policy': 5, 'state': 6, 'public authorities': 7, 'taxes': 8, 'transport': 9, 'science': 10, 'research and technology': 11, 'european union': 12, 'work and employment': 13, 'health': 14, 'education': 15, 'industry': 16, 'sports': 17}


In [38]:
from transformers import RobertaForSequenceClassification, RobertaConfig

In [39]:
num_labels = len(unique_labels)

In [40]:
model = RobertaForSequenceClassification.from_pretrained(notebook_path_prefix+"_lm", num_labels=num_labels)

loading configuration file roberta_lm_lm\config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,
    "LABEL_17": 17,
    "LABEL_2":

In [41]:
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions >= 0.5).astype(int) #.argmax(-1)
    
    # print(labels, preds)
    
    # try:
    acc = accuracy_score(labels, preds)
    # except ValueError:
    
    
    return {
        'accuracy': acc,
        'f1': f1_score(y_true=labels, y_pred=preds, average='weighted'),
        'precision': precision_score(y_true=labels, y_pred=preds, average='weighted'),
        'recall': recall_score(y_true=labels, y_pred=preds, average='weighted')
    }

In [42]:
import numpy as np

In [43]:
from transformers import  Trainer, TrainingArguments

In [44]:
training_args = TrainingArguments(
    output_dir=notebook_path_prefix+"_classification",
    warmup_steps=500,
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=17,
    per_device_eval_batch_size=30,
    save_steps=5_000,
    save_total_limit=3,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    logging_steps=700,
    eval_steps=700,
    evaluation_strategy='steps',
    report_to="wandb",
    run_name="roberta-classification"
)

PyTorch: setting up devices


In [45]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_ds,         # training dataset
    eval_dataset=dev_ds,            # evaluation dataset
    compute_metrics=compute_metrics
)

In [46]:
trainer.evaluate(test_ds)

***** Running Evaluation *****
  Num examples = 6858
  Batch size = 30


C:\Anaconda3\envs\PetraRQ2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 0.7199797034263611,
 'eval_accuracy': 0.00014581510644502772,
 'eval_f1': 0.0002490339082464446,
 'eval_precision': 0.0004896459450760735,
 'eval_recall': 0.00019462826002335538,
 'eval_runtime': 264.9451,
 'eval_samples_per_second': 25.885,
 'eval_steps_per_second': 0.864}

In [47]:
trainer.train()

***** Running training *****
  Num examples = 24003
  Num Epochs = 5
  Instantaneous batch size per device = 17
  Total train batch size (w. parallel, distributed & accumulation) = 17
  Gradient Accumulation steps = 1
  Total optimization steps = 7060


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
700,0.261600,0.104666,0.625547,0.678703,0.813450,0.599293
1400,0.099900,0.071193,0.718868,0.812302,0.915701,0.742358
2100,0.071200,0.060752,0.750948,0.840987,0.918501,0.785402
2800,0.063000,0.053062,0.790610,0.870800,0.926771,0.826159
3500,0.046900,0.051354,0.801108,0.880880,0.925592,0.844874
4200,0.043200,0.048688,0.808107,0.887665,0.914598,0.865253
4900,0.033400,0.046776,0.821814,0.894140,0.925298,0.867332
5600,0.031200,0.046743,0.815981,0.892067,0.925757,0.863589
6300,0.024800,0.046144,0.823564,0.896569,0.931438,0.867748
7000,0.023700,0.045806,0.825022,0.896890,0.930830,0.868788


***** Running Evaluation *****
  Num examples = 3429
  Batch size = 30
C:\Anaconda3\envs\PetraRQ2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 3429
  Batch size = 30
C:\Anaconda3\envs\PetraRQ2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 3429
  Batch size = 30
C:\Anaconda3\envs\PetraRQ2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predi

TrainOutput(global_step=7060, training_loss=0.06947880826320595, metrics={'train_runtime': 7899.7824, 'train_samples_per_second': 15.192, 'train_steps_per_second': 0.894, 'total_flos': 9395897647933440.0, 'train_loss': 0.06947880826320595, 'epoch': 5.0})

In [48]:
trainer.evaluate(test_ds)

***** Running Evaluation *****
  Num examples = 6858
  Batch size = 30


C:\Anaconda3\envs\PetraRQ2\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.05427917465567589,
 'eval_accuracy': 0.794546515018956,
 'eval_f1': 0.8889033891770258,
 'eval_precision': 0.9228076499900962,
 'eval_recall': 0.8590891397430906,
 'eval_runtime': 303.3825,
 'eval_samples_per_second': 22.605,
 'eval_steps_per_second': 0.755,
 'epoch': 5.0}